In [ ]:
import cv2
import numpy as np

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        cv2.fillPoly(mask, vertices, ignore_mask_color)
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, a=0.8, b=1., c=0.):
    return cv2.addWeighted(initial_img, a, img, b, c)

cap = cv2.VideoCapture("Drive.mp4")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # Apply image processing pipeline
        gray = grayscale(frame)
        blur = gaussian_blur(gray, 5)
        edges = canny(blur, 50, 150)
        imshape = frame.shape
        vertices = np.array([[(0,imshape[0]),(450, 320), (490, 320), (imshape[1],imshape[0])]], dtype=np.int32)
        masked_edges = region_of_interest(edges, vertices)
        line_image = hough_lines(masked_edges, 2, np.pi/180, 15, 40, 20)
        result = weighted_img(line_image, frame, 0.8, 1., 0.)
        
        # Write the processed frame to output video
        out.write(result)
        
        # Display the result
        cv2.imshow('result', result)
        
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

#### 영상 중간에 NoneType에러가 발생하여 영상이 멈춰버리는 현상이 생김.

In [ ]:
def filter_colors(image):
    # Filter white pixels
    lower_white = np.array([200, 200, 200])
    upper_white = np.array([255, 255, 255])
    white_mask = cv2.inRange(image, lower_white, upper_white)
    white_image = cv2.bitwise_and(image, image, mask=white_mask)

    # Filter yellow pixels
    lower_yellow = np.array([180, 180, 0])
    upper_yellow = np.array([255, 255, 180])
    yellow_mask = cv2.inRange(image, lower_yellow, upper_yellow)
    yellow_image = cv2.bitwise_and(image, image, mask=yellow_mask)

    # Combine the two filtered images
    image2 = cv2.addWeighted(white_image, 1., yellow_image, 1., 0)
    return image2

#### 차선 중 흰선과 노란선을 인식하는 코드를 만들었다. 이중 lower와 upper를 이용해 색상의 범위를 잡아줘서
#### 어느정도 어두워도 인식할 수 있게 만들어 보았습니다.

# 2차 완성코드  nonetype오류

In [ ]:
import cv2
import numpy as np

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

def filter_colors(image):
    # Filter white pixels
    lower_white = np.array([200, 200, 200])
    upper_white = np.array([255, 255, 255])
    white_mask = cv2.inRange(image, lower_white, upper_white)
    white_image = cv2.bitwise_and(image, image, mask=white_mask)

    # Filter yellow pixels
    lower_yellow = np.array([180, 180, 0])
    upper_yellow = np.array([255, 255, 180])
    yellow_mask = cv2.inRange(image, lower_yellow, upper_yellow)
    yellow_image = cv2.bitwise_and(image, image, mask=yellow_mask)

    # Combine the two filtered images
    image2 = cv2.addWeighted(white_image, 1., yellow_image, 1., 0)
    return image2

def process_image(image):
    # Define vertices for the region of interest
    imshape = image.shape
    
    vertices = np.array([[(0,imshape[0]),(450, 320), (490, 320), (imshape[1]-150,imshape[0])]], dtype=np.int32)

    # Apply color filtering
    filtered_image = filter_colors(image)

    # Convert to grayscale
    gray = grayscale(filtered_image)

    # Apply Gaussian smoothing
    blur = gaussian_blur(gray, kernel_size=5)

    # Apply Canny edge detection
    edges = canny(blur, low_threshold=50, high_threshold=150)
    # Apply region of interest mask
    masked_edges = region_of_interest(edges, vertices)

    # Apply Hough transform to detect lines
    lines = hough_lines(masked_edges, rho=1, theta=np.pi/180, threshold=20, min_line_len=20, max_line_gap=300)

    # Draw lines on original image
    line_image = np.zeros_like(image)
    draw_lines(line_image, lines)
    final_image = cv2.addWeighted(image, 0.8, line_image, 1, 0)

    return final_image

cap = cv2.VideoCapture("Drive.mp4")
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        result = process_image(frame)
        cv2.imshow('result', result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

#### 동영상 실행을 해보았고 나갈 때 까지 잡음도 어느 정도 잡아줘서 문제 없이 진행하던 중 영상 후반부에서 다시 TypeError: 'NoneType' object is not iterable 오류가 발생하였습니다.

이 문제를 해결하기 위해서 검색해보면서 해결방법을 찾아봤었는데
TypeError: 'NoneType' object is not iterable 오류는 객체의 타입이 None일 때, 반복 가능한(iterable) 객체로 사용하려 할 때 발생하는 에러입니다.

주어진 코드에서 해당 에러가 발생하는 부분은 draw_lines 함수 내부의 for 반복문입니다. 해당 반복문에서 lines 변수가 None일 경우, for 반복문에서 이를 반복 가능한 객체로 사용하려 하기 때문에 에러가 발생합니다.

이 경우, hough_lines 함수에서 cv2.HoughLinesP 함수의 반환값이 None일 수 있습니다. 따라서 hough_lines 함수에서 반환값이 None인 경우를 처리하는 로직을 추가해야 합니다. 반환값이 None인 경우에는 draw_lines 함수를 호출하지 않고 함수를 종료하면 됩니다.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        cv2.fillPoly(mask, vertices, ignore_mask_color)
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, a=0.8, b=1., c=0.):
    return cv2.addWeighted(initial_img, a, img, b, c)

cap = cv2.VideoCapture("Drive.mp4")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output2.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # Apply image processing pipeline
        gray = grayscale(frame)
        blur = gaussian_blur(gray, 5)
        edges = canny(blur, 50, 150)
        imshape = frame.shape
        vertices = np.array([[(210, imshape[0]), (300, 300), (420, 300), (350, imshape[0])],
                             [(550, imshape[0]), (640, 320), (660, 320), (700, imshape[0])]], dtype=np.int32)
        #vertices = np.array([[(0,imshape[0]),(450, 320), (490, 320), (imshape[1],imshape[0])],
                            #[(0,imshape[0]),(450, 320), (490, 320), (imshape[1],imshape[0])]], dtype=np.int32)
        masked_edges = region_of_interest(edges, vertices)
        line_image = hough_lines(masked_edges, 2, np.pi/180, 15, 40, 20)
        result = weighted_img(line_image, frame, 0.8, 1., 0.)
        
        # Write the processed frame to output video
        out.write(result)
        
        # Display the result
        cv2.imshow('result', result)
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):

            break


# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
정점 = np.array([[(150, imshape[0]), (300, 360), (420, 360), (imshape[1]-500, imshape[0])], [(600, imshape[ 0]), (480, 360), (550, 360), (imshape[1]-130, imshape[0])]], dtype=np.int32) 값을 얻지 못했을 때 실행 시 'NoneType' object is not iterable 오류가 발생하면 이유가 활성화됩니까?

### -------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import cv2
import numpy as np

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    if lines is None:
        return []
    return lines
def filter_colors(image):
    # Filter white pixels
    lower_white = np.array([200, 200, 200])
    upper_white = np.array([255, 255, 255])
    white_mask = cv2.inRange(image, lower_white, upper_white)
    white_image = cv2.bitwise_and(image, image, mask=white_mask)

    # Filter yellow pixels
    lower_yellow = np.array([180, 180, 0])
    upper_yellow = np.array([255, 255, 180])
    yellow_mask = cv2.inRange(image, lower_yellow, upper_yellow)
    yellow_image = cv2.bitwise_and(image, image, mask=yellow_mask)

    # Combine the two filtered images
    image2 = cv2.addWeighted(white_image, 1., yellow_image, 1., 0)
    return image2

def process_image(image):
    # Define vertices for the region of interest
    imshape = image.shape
    
    vertices = np.array([[(600,imshape[0]-280),
                     (900,imshape[0]-450),
                     (1100,imshape[0]-450),
                     (1400, imshape[0]-280)]], dtype=np.int32)
    #vertices = np.array([[(0,imshape[0]),(450, 320), (490, 320), (imshape[1]-150,imshape[0])]], dtype=np.int32)
    #vertices = np.array([[(150, imshape[0]), (300, 360), (420, 360), (imshape[1]-500, imshape[0])],
    #                         [(600, imshape[0]), (480, 360), (550, 360), (imshape[1]-130, imshape[0])]], dtype=np.int32)
    # Apply color filtering
    filtered_image = filter_colors(image)

    # Convert to grayscale
    gray = grayscale(filtered_image)

    # Apply Gaussian smoothing
    blur = gaussian_blur(gray, kernel_size=5)

    # Apply Canny edge detection
    edges = canny(blur, low_threshold=50, high_threshold=150)
    # Apply region of interest mask
    masked_edges = region_of_interest(edges, vertices)

    # Apply Hough transform to detect lines
    lines = hough_lines(masked_edges, rho=1, theta=np.pi/180, threshold=20, min_line_len=20, max_line_gap=300)

    # Draw lines on original image
    line_image = np.zeros_like(image)
    draw_lines(line_image, lines)
    final_image = cv2.addWeighted(image, 0.8, line_image, 1, 0)

    return final_image

cap = cv2.VideoCapture("MDR_221009_024928.AVI")
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        result = process_image(frame)
        cv2.imshow('result', result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.imwrite('frame4.jpg', frame)
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

환경 조건을 적용하려면, 먼저 어떤 환경 조건을 적용할지 결정해야 합니다. 예를 들어, 날씨가 흐릴 때 이미지 처리 파이프라인에서 사용하는 컬러 필터링이 제대로 작동하지 않을 수 있으므로, 다른 컬러 필터링 방법을 적용하는 것이 좋을 수 있습니다.

이를 위해 filter_colors() 함수를 수정하여, 날씨 조건에 따라 적절한 필터링을 적용할 수 있습니다. 예를 들어, 날씨가 흐릴 때는 밝기 값이 작아질 수 있으므로, 밝기를 낮추는 등의 필터링 방법을 추가로 적용할 수 있습니다.

또한, 이미지 처리 파이프라인 전체를 동적으로 재조정할 수도 있습니다. 예를 들어, 날씨 조건이 흐릴 때는 Canny edge detection에서 사용하는 low_threshold 값을 더 작게 조정하여 엣지 검출이 더 잘되도록 할 수 있습니다.

이 외에도, 다양한 환경 조건에 따라 파이프라인을 동적으로 재조정하는 방법이 있을 수 있으니, 구체적인 환경 조건과 파이프라인 구조에 따라 적절한 조정을 진행하시면 됩니다.

In [2]:
def slide_window_search(binary_warped, left_current, right_current):
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))

    nwindows = 4
    window_height = np.int(binary_warped.shape[0] / nwindows)
    nonzero = binary_warped.nonzero()  # 선이 있는 부분의 인덱스만 저장 
    nonzero_y = np.array(nonzero[0])  # 선이 있는 부분 y의 인덱스 값
    nonzero_x = np.array(nonzero[1])  # 선이 있는 부분 x의 인덱스 값 
    margin = 100
    minpix = 50
    left_lane = []
    right_lane = []
    color = [0, 255, 0]
    thickness = 2

    for w in range(nwindows):
        win_y_low = binary_warped.shape[0] - (w + 1) * window_height  # window 윗부분
        win_y_high = binary_warped.shape[0] - w * window_height  # window 아랫 부분
        win_xleft_low = left_current - margin  # 왼쪽 window 왼쪽 위
        win_xleft_high = left_current + margin  # 왼쪽 window 오른쪽 아래
        win_xright_low = right_current - margin  # 오른쪽 window 왼쪽 위 
        win_xright_high = right_current + margin  # 오른쪽 window 오른쪽 아래

        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high), color, thickness)
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), color, thickness)
        good_left = ((nonzero_y >= win_y_low) & (nonzero_y < win_y_high) & (nonzero_x >= win_xleft_low) & (nonzero_x < win_xleft_high)).nonzero()[0]
        good_right = ((nonzero_y >= win_y_low) & (nonzero_y < win_y_high) & (nonzero_x >= win_xright_low) & (nonzero_x < win_xright_high)).nonzero()[0]
        left_lane.append(good_left)
        right_lane.append(good_right)
        # cv2.imshow("oo", out_img)

        if len(good_left) > minpix:
            left_current = np.int(np.mean(nonzero_x[good_left]))
        if len(good_right) > minpix:
            right_current = np.int(np.mean(nonzero_x[good_right]))

    left_lane = np.concatenate(left_lane)  # np.concatenate() -> array를 1차원으로 합침
    right_lane = np.concatenate(right_lane)

    leftx = nonzero_x[left_lane]
    lefty = nonzero_y[left_lane]
    rightx = nonzero_x[right_lane]
    righty = nonzero_y[right_lane]

    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
    left_fitx = left_fit[0] * ploty ** 2 + left_fit[1] * ploty + left_fit[2]
    right_fitx = right_fit[0] * ploty ** 2 + right_fit[1] * ploty + right_fit[2]

    ltx = np.trunc(left_fitx)  # np.trunc() -> 소수점 부분을 버림
    rtx = np.trunc(right_fitx)

    out_img[nonzero_y[left_lane], nonzero_x[left_lane]] = [255, 0, 0]
    out_img[nonzero_y[right_lane], nonzero_x[right_lane]] = [0, 0, 255]

    plt.imshow(out_img)
    plt.plot(left_fitx, ploty, color = 'yellow')
    plt.plot(right_fitx, ploty, color = 'yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.show()

    ret = {'left_fitx' : ltx, 'right_fitx': rtx, 'ploty': ploty}

    return ret

## 과제 1차-------------------------------------------------------------------------------------------------------------------

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg



def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        cv2.fillPoly(mask, vertices, ignore_mask_color)
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, a=0.8, b=1., c=0.):
    return cv2.addWeighted(initial_img, a, img, b, c)


cap = cv2.VideoCapture("MDR_221009_024928.AVI")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output2.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # Apply image processing pipeline
        gray = grayscale(frame)
        blur = gaussian_blur(gray, 5)
        edges = canny(blur, 50, 150)
        imshape = frame.shape
        vertices = np.array([[(600,imshape[0]-280),(900,imshape[0]-450),(1100,imshape[0]-450),(1400, imshape[0]-280)]], dtype=np.int32)
        
        vertices2 = np.array([[(600,imshape[0]-280),(900,imshape[0]-450),(950,imshape[0]-450),(800,imshape[0]-280)],
                     [(1100,imshape[0]-280),(1050,imshape[0]-450),(1080,imshape[0]-450),(1250, imshape[0]-280)]], dtype=np.int32)
        masked_edges = region_of_interest(edges, vertices2)
        line_image = hough_lines(masked_edges, 2, np.pi/180, 15, 40, 20)
        result = weighted_img(line_image, frame, 0.8, 1., 0.)
        
        # Write the processed frame to output video
        out.write(result)
        
        # Display the result
        cv2.imshow('result', result)
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):

            break


# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

## 수정 1차----------------------------------------------------------------------------------------------

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg



def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        cv2.fillPoly(mask, vertices, ignore_mask_color)
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image

def draw_lines(img, lines, color=[0, 255, 0], thickness=5):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, a=0.8, b=1., c=0.):
    return cv2.addWeighted(initial_img, a, img, b, c)


cap = cv2.VideoCapture("MDR_221009_024928.AVI")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output2.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # Apply image processing pipeline
        gray = grayscale(frame)
        blur = gaussian_blur(gray, 5)
        edges = canny(blur, low_threshold=50, high_threshold=150)
        imshape = frame.shape
        vertices = np.array([[(600,imshape[0]-280),(900,imshape[0]-450),(1100,imshape[0]-450),(1400, imshape[0]-280)]], dtype=np.int32)
        
        vertices2 = np.array([[(600,imshape[0]-280),(900,imshape[0]-450),(950,imshape[0]-450),(800,imshape[0]-290)],
                     [(1100,imshape[0]-280),(1050,imshape[0]-450),(1080,imshape[0]-450),(1250, imshape[0]-280)]], dtype=np.int32)
        masked_edges = region_of_interest(edges, vertices2)
        line_image = hough_lines(masked_edges, 2, np.pi/180, 15, 40, 20)
        result = weighted_img(line_image, frame, 0.8, 1., 0.)
        
        # Write the processed frame to output video
        out.write(result)
        
        # Display the result
        cv2.imshow('result', result)
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

### ----------------------------------------------------------------------------------------------------------------------------

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg



def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, a=0.8, b=1., c=0.):
    return cv2.addWeighted(initial_img, a, img, b, c)


def select_white_yellow(img):
    # Convert to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

    # Define range of white color in HSV
    lower_white = np.array([0, 0, 200], dtype=np.uint8)
    upper_white = np.array([255, 30, 255], dtype=np.uint8)

    # Define range of yellow color in HSV
    lower_yellow = np.array([20, 100, 100], dtype=np.uint8)
    upper_yellow = np.array([30, 255, 255], dtype=np.uint8)

    # Threshold the HSV image to get only white and yellow colors
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    mask = cv2.bitwise_or(mask_white, mask_yellow)
    masked_img = cv2.bitwise_and(img, img, mask=mask)

    return masked_img

cap = cv2.VideoCapture("MDR_221009_024928.AVI")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output2.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # Apply image processing pipeline
        selected = select_white_yellow(frame)
        gray = grayscale(selected)
        blur = gaussian_blur(gray, 5)
        edges = canny(blur, 50, 150)
        imshape = frame.shape
        vertices = np.array([[(600,imshape[0]-280),(900,imshape[0]-450),(1100,imshape[0]-450),(1400, imshape[0]-280)]], dtype=np.int32)
        
        vertices2 = np.array([[(600,imshape[0]-280),(900,imshape[0]-450),(950,imshape[0]-450),(800,imshape[0]-280)],
                     [(1100,imshape[0]-280),(1050,imshape[0]-450),(1080,imshape[0]-450),(1250, imshape[0]-280)]], dtype=np.int32)
        masked_edges = region_of_interest(edges, vertices2)
        line_image = hough_lines(masked_edges, 2, np.pi/180, 15, 40, 20)
        result = weighted_img(line_image, frame, 0.8, 1., 0.)
        
        # Write the processed frame to output video
        out.write(result)
        ㅂ
        # Display the result
        cv2.imshow('result', result)
        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):

            break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()